In [ ]:
import librosa 
import librosa.display
import IPython.display as ipd
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
import os
import math
import pathlib
import seaborn as sns
import tensorflow as tf
import tensorflow.keras as keras

import json
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers
from tensorflow.keras import models
from IPython import display

In [ ]:
# Set the seed value for experiment reproducibility.
seed = 42
tf.random.set_seed(seed)
np.random.seed(seed)
DATASET_PATH = "/content/drive/MyDrive/Data/genres_original"
print(list(os.walk(DATASET_PATH)))
path_list = list(os.walk(DATASET_PATH))
path_list.remove(path_list[0])
path_list.remove(path_list[9])
for (dirpath, dirnames, filenames) in path_list:
  print(f"Dirpath: {dirpath}")
  print(f"Dirname: {dirnames}")
  print(f"Filenames: {filenames}")
  print()

In [ ]:

DATASET_PATH = "/content/drive/MyDrive/Data/genres_original"
JSON_PATH = "data_10.json"
SAMPLE_RATE = 22050
TRACK_DURATION = 30 # measured in seconds
SAMPLES_PER_TRACK = SAMPLE_RATE * TRACK_DURATION


def save_mfcc(dataset_path, json_path, num_mfcc=13, n_fft=2048, hop_length=512, num_segments=5):
    """Extracts MFCCs from music dataset and saves them into a json file along witgh genre labels.
        :param dataset_path (str): Path to dataset
        :param json_path (str): Path to json file used to save MFCCs
        :param num_mfcc (int): Number of coefficients to extract
        :param n_fft (int): Interval we consider to apply FFT. Measured in # of samples
        :param hop_length (int): Sliding window for FFT. Measured in # of samples
        :param: num_segments (int): Number of segments we want to divide sample tracks into
        :return:
        """

    # dictionary to store mapping, labels, and MFCCs
    data = {
        "mapping": [],
        "labels": [],
        "mfcc": []
    }

    samples_per_segment = int(SAMPLES_PER_TRACK / num_segments)
    num_mfcc_vectors_per_segment = math.ceil(samples_per_segment / hop_length)

    # loop through all genre sub-folder
    for i, (dirpath, dirnames, filenames) in enumerate(path_list):

        # ensure we're processing a genre sub-folder level
        if dirpath is not dataset_path:

            # save genre label (i.e., sub-folder name) in the mapping
            semantic_label = dirpath.split("/")[-1]
            data["mapping"].append(semantic_label)
            print("\nProcessing: {}".format(semantic_label))

            # process all audio files in genre sub-dir
            for f in filenames:

		# load audio file
                file_path = os.path.join(dirpath, f)
                signal, sample_rate = librosa.load(file_path, sr=SAMPLE_RATE)
                RMS=math.sqrt(np.mean(signal**2))
                noise=np.random.normal(0, RMS, signal.shape[0])
                signal = signal+noise


                # process all segments of audio file
                for d in range(num_segments):

                    # calculate start and finish sample for current segment
                    start = samples_per_segment * d
                    finish = start + samples_per_segment
                    # extract mfcc
                    print(f"Signal before: {signal[start:finish]}")
                    mfcc = librosa.feature.mfcc(signal[start:finish], sample_rate, n_mfcc=num_mfcc, n_fft=n_fft, hop_length=hop_length)
                    print(f"MFCC after: {mfcc}")
                    print(f"MFCC shape: {mfcc.shape}")
                    mfcc = mfcc.T

                    # store only mfcc feature with expected number of vectors
                    if len(mfcc) == num_mfcc_vectors_per_segment:
                        data["mfcc"].append(mfcc.tolist())
                        data["labels"].append(i)
                        print("{}, segment:{}".format(file_path, d+1))

    # save MFCCs to json file
    with open(json_path, "w") as fp:
        json.dump(data, fp, indent=4)
        

In [ ]:
save_mfcc(DATASET_PATH,JSON_PATH,num_segments=5)

In [ ]:
def load_data(data_path):
    """Loads training dataset from json file.
        :param data_path (str): Path to json file containing data
        :return X (ndarray): Inputs
        :return y (ndarray): Targets
    """

    with open(data_path, "r") as fp:
        data = json.load(fp)

    # convert lists to numpy arrays
    X = np.array(data["mfcc"])
    y = np.array(data["labels"])

    print("Data succesfully loaded!")

    return  X, y

In [ ]:
DATA_PATH = "/content/data_10.json"

def prepare_datasets(test_size, validation_size):
    """Loads data and splits it into train, validation and test sets.
    :param test_size (float): Value in [0, 1] indicating percentage of data set to allocate to test split
    :param validation_size (float): Value in [0, 1] indicating percentage of train set to allocate to validation split
    :return X_train (ndarray): Input training set
    :return X_validation (ndarray): Input validation set
    :return X_test (ndarray): Input test set
    :return y_train (ndarray): Target training set
    :return y_validation (ndarray): Target validation set
    :return y_test (ndarray): Target test set
    """

    # load data
    X, y = load_data(DATA_PATH)

    # create train, validation and test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size)
    X_train, X_validation, y_train, y_validation = train_test_split(X_train, y_train, test_size=validation_size)

    # add an axis to input sets
    X_train = X_train[..., np.newaxis]
    X_validation = X_validation[..., np.newaxis]
    X_test = X_test[..., np.newaxis]

    return X_train, X_validation, X_test, y_train, y_validation, y_test

In [ ]:
X_train, X_validation, X_test, y_train, y_validation, y_test = prepare_datasets(
...     0.25, 0.2)

In [ ]:
X.shape

In [ ]:
np.count_nonzero(targets==10)

In [ ]:
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(inputs.shape[1],inputs.shape[2])),
    keras.layers.Dense(512,activation="relu", kernel_regularizer=keras.regularizers.l2(0.001)),
     keras.layers.Dropout(0.3),
    keras.layers.Dense(256,activation="relu",kernel_regularizer=keras.regularizers.l2(0.001)),
     keras.layers.Dropout(0.3),
    keras.layers.Dense(64,activation="relu", kernel_regularizer=keras.regularizers.l2(0.001)), 
     keras.layers.Dropout(0.3),   
    keras.layers.Dense(10,activation="softmax", kernel_regularizer=keras.regularizers.l2(0.001))

])

In [ ]:
input_shape = (X_train.shape[1], X_train.shape[2], 1)
model = keras.Sequential([
 keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
 keras.layers.MaxPooling2D((3, 3), strides=(2, 2), padding='same'),
 keras.layers.BatchNormalization(),

    # 2nd conv layer
 keras.layers.Conv2D(32, (3, 3), activation='relu'),
 keras.layers.MaxPooling2D((3, 3), strides=(2, 2), padding='same'),
 keras.layers.BatchNormalization(),

    # 3rd conv layer
 keras.layers.Conv2D(32, (2, 2), activation='relu'),
 keras.layers.MaxPooling2D((2, 2), strides=(2, 2), padding='same'),
 keras.layers.BatchNormalization(),

    # flatten output and feed it into dense layer
 keras.layers.Flatten(),
 keras.layers.Dense(64, activation='relu'),
 keras.layers.Dropout(0.3),

    # output layer
 keras.layers.Dense(10, activation='softmax')
 ])

In [ ]:
optimizer = keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer, loss=tf.keras.losses.SparseCategoricalCrossentropy(), metrics=["accuracy"])

In [ ]:
model.summary() 

In [ ]:
history = model.fit(X_train, y_train, validation_data=(X_validation, y_validation), batch_size=32, epochs=60)

In [ ]:
def plot_history(history):
    """Plots accuracy/loss for training/validation set as a function of the epochs
        :param history: Training history of model
        :return:
    """

    fig, axs = plt.subplots(2)

    # create accuracy sublpot
    axs[0].plot(history.history["accuracy"], label="train accuracy")
    axs[0].plot(history.history["val_accuracy"], label="test accuracy")
    axs[0].set_ylabel("Accuracy")
    axs[0].legend(loc="lower right")
    axs[0].set_title("Accuracy eval")

    # create error sublpot
    axs[1].plot(history.history["loss"], label="train error")
    axs[1].plot(history.history["val_loss"], label="test error")
    axs[1].set_ylabel("Error")
    axs[1].set_xlabel("Epoch")
    axs[1].legend(loc="upper right")
    axs[1].set_title("Error eval")

    plt.show()

In [ ]:
model.save("/content/music_genre_classification_model.h5")
new_model = keras.models.load_model('/content/music_genre_classification_model.h5')

# Check its architecture
new_model.summary()

In [ ]:
plot_history(history)

In [ ]:
X.shape

In [ ]:
    # pick a sample to predict from the test set
X_to_predict = X_test[100]
y_to_predict = y_test[100]

X_test.shape
    # predict sample
X_to_predict = X_to_predict[np.newaxis,...]
prediction = model.predict(X_to_predict)

In [ ]:
X_test.shape

In [ ]:
X_test[100].shape

In [ ]:
predicted_index = np.argmax(prediction)
predicted_index

In [ ]:
y_to_predict